In [24]:
import os
import librosa
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
import parselmouth 
from parselmouth.praat import call
import cmudict
import regex as re
import ffmpeg
import shutil
nltk.download('cmudict')
plus = pd.read_csv('mustard++_text.csv')

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\kasia\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


In [25]:
plus = plus[plus['Sarcasm'].notna()]
pd.set_option('display.max_colwidth', None)
plus['wrd_count'] = plus['SENTENCE'].astype(str).apply(lambda x: len(x.split())) # determine max word count
plus = plus[plus['wrd_count'] <= 20] #max word count 20
plus.loc[plus['SENTENCE'].str.contains('?', regex=False), 'QUESTION?'] = True #is this utterance a Q?
plus['QUESTION?'] = plus['QUESTION?'].fillna(False)
plus = plus.infer_objects(copy=False)
plus = plus[plus['SCENE'] != '1_1001']
plus['SCENE'] = plus['SCENE'].astype(str) + '_u'
plus = plus.drop(columns=['KEY','Sarcasm_Type'])
spee = ['RAJ', 'DINESH', 'MODERATOR', 'MEMBER-GIRL','MEMBER-BOY']
plus = plus[~plus['SPEAKER'].isin(spee)]
meen = plus["SPEAKER"].isin([
    'SHELDON','HOWARD','LEONARD','STUART',
    'CHANDLER','JOEY','ROSS','GILFOYLE',
    'RICHARD','ERLICH'
])
print(meen.value_counts())
plus["SPEAKER"] = plus["SPEAKER"].str.strip().str.upper()
plus["GENDER"] = meen.map({True: "M", False: "F"})

plus = plus.reset_index()

SPEAKER
True     591
False    432
Name: count, dtype: int64


C:\Users\kasia\AppData\Local\Temp\ipykernel_31552\4138947182.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  plus['QUESTION?'] = plus['QUESTION?'].fillna(False)


In [26]:
what = plus[plus['SCENE'].str.contains("1_S10E01_225_u")]
what

,index,SCENE,SENTENCE,END_TIME,SPEAKER,SHOW,Sarcasm,Implicit_Emotion,Explicit_Emotion,Valence,Arousal,wrd_count,QUESTION?,GENDER
268,2046,1_S10E01_225_u,"Well, he probably buys his illegal drugs from a local vendor.",0:3.5,OTHER,BBT,1.0,Ridicule,Neutral,3.0,8.0,11,False,F


In [27]:
men = plus["SCENE"].isin([
    '1_4639_u','1_4789_u','1_7494_u','1_7504_u',
    '1_7953_u','1_8136_u','1_9963_u','1_11051_u',
    '1_11885_u','2_55_u','2_150_u','2_193_u','2_228_u',
    '2_245_u','2_278_u','2_295_u','2_341_u','2_415_u',
    '2_530_u','2_548_u','2_609_u','2_627_u','1_6645_u',
    '2_42_u','1_S10E01_225_u','1_S11E08_351_u','1_S12E02_274_u',
    '1_S12E04_204_u','1_S12E04_233_u','1_S12E11_072_u','1_S12E20_378_u',
    '3_S01E02_461_u','3_S01E04_327_u'
])
print(men.value_counts())
plus["SPEAKER"] = plus["SPEAKER"].str.strip().str.upper()
plus["GENDER"] = men.map({True: "M", False: "F"})


plus = plus.reset_index()

SCENE
False    990
True      33
Name: count, dtype: int64


In [28]:
print(len(plus))

1023


In [29]:
root_folder = r"C:\Users\kasia\asp2025\textgrids"
wav_files = []
for dirpath, _, filenames in os.walk(root_folder):
    for f in filenames:
        if f.lower().endswith(".wav"):
            wav_files.append(f)

wav_set = set(wav_files)
wav_set = {os.path.splitext(f)[0] for f in wav_set}
plus = plus[plus["SCENE"].str.strip().isin(wav_set)]

In [30]:
print(len(plus))

980


In [31]:
plus = plus.drop(columns=['index'])
plus.head(7)

,level_0,SCENE,SENTENCE,END_TIME,SPEAKER,SHOW,Sarcasm,Implicit_Emotion,Explicit_Emotion,Valence,Arousal,wrd_count,QUESTION?,GENDER
0,0,1_10004_u,"And of those few months, how long have you been a demented sex pervert?",0:07,SHELDON,BBT,0.0,Anger,Anger,2.0,7.0,14,True,F
1,1,1_10009_u,"Let the dead man talk. So, why do you think that?",0:05,PENNY,BBT,0.0,Happiness,Happiness,6.0,5.0,11,True,F
2,2,1_10495_u,"And if it turns out you don't... - you and I could live together. - You and... Oh, sure.",0:07,AMY,BBT,1.0,Disgust,Neutral,3.0,7.0,19,False,F
3,3,1_105_u,I'm just inferring this is a couch because the evidence suggests the coffee table is having a tiny garage sale.,0:06,SHELDON,BBT,1.0,Disgust,Neutral,2.0,6.0,20,False,F
4,4,1_10748_u,"There they go, fighting again. You'd never hear her talk that way to Sauvignon Blanc.",0:07,AMY,BBT,1.0,Disgust,Sadness,3.0,7.0,15,False,F
5,5,1_10797_u,"I'm with him three years, nothing. She's with two minutes, and he's taking his pants off.",0:07,AMY,BBT,0.0,Frustration,Frustration,4.0,6.0,16,False,F
6,6,1_10829_u,"On the bright side, that Oxford study was right. One friend down. I wonder who you're going to lose next.",0:1,SHELDON,BBT,1.0,Disgust,Happiness,3.0,4.0,20,False,F


In [37]:
haha = (plus['GENDER'].str.contains('F')) & (plus['Sarcasm'] == 0)
haha.sum()

0     True
1     True
2    False
3    False
4    False
dtype: bool

In [ ]:
len(plus['SPEAKER'].unique()) #find unique val in col

In [ ]:
plus['SPEAKER'].unique()

In [ ]:
len(plus[plus['SPEAKER'].str.contains('ROSS')]) #double check if smth exists

In [ ]:
(plus["GENDER"] == "F").sum()

In [ ]:
mask = plus["COL_A"].str.contains("foo") & plus["COL_B"].str.contains("bar")
count = mask.sum()
print(count)


In [ ]:
haha = plus['GENDER'].str.contains('F') & plus['Sarcasm'] == 0
print(haha.sum())

In [ ]:
who = plus[plus['QUESTION?'] == True]
who

In [ ]:
len(who)

In [ ]:
len(who[who['Sarcasm']==1])

In [ ]:
what = plus[plus['SCENE'].str.contains("1_S10E10_269")]
what

In [ ]:
#plus.iloc[526:546]

In [ ]:
#count num files in folder
file_count = sum(len(files) for _, _, files in os.walk(r'C:\Users\kasia\asp2025\textgrids'))
print(file_count)



# BASELINE FEATURES

In [ ]:
# source = https://github.com/drfeinberg/PraatScripts/tree/master

plus['pitchmean'] = None
plus['pitchrange'] = None
plus['pitchsd'] = None
plus['dbmean'] = None
plus['dbrange'] = None
plus['dbsd'] = None
def measurePitch(voiceID, f0min, f0max, unit):
    duration = call(sound, "Get total duration") # duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) # get standard deviation
    
    return duration, meanF0, stdevF0

def measurePitch(sound, f0min, f0max, unit):
    duration = call(sound, "Get total duration")
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit)
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit)
    minF0 = call(pitch, "Get minimum", 0, 0, unit, "Parabolic") #check this
    maxF0 = call(pitch, "Get maximum", 0, 0, unit, "Parabolic")
    f0range = maxF0 - minF0
    intensity = sound.to_intensity(time_step=0.01) #huh

    meanDb = call(intensity, "Get mean", 0, 0, "energy")
    stdevDb = call(intensity, "Get standard deviation", 0, 0)
    minDb = call(intensity, "Get minimum", 0, 0, "Parabolic")
    maxDb = call(intensity, "Get maximum", 0, 0, "Parabolic")
    dbrange = maxDb - minDb

    return duration, meanF0, stdevF0, f0range, meanDb, stdevDb, dbrange

base_dir = r"C:\Users\kasia\asp2025\textgrids"
for i, key in enumerate(plus['SCENE']):
    try:
        wav_path = None
        for root, dirs, files in os.walk(base_dir):
            target_file = f"{key}_u.wav"
            if target_file in files:
                wav_path = os.path.join(root, target_file)
                break  

        if wav_path is None:
            raise FileNotFoundError(f"{key}_u.wav not found in subfolders")

        sound = parselmouth.Sound(wav_path)
        (duration, meanF0, stdevF0, f0range, meanDb, stdevDb, dbrange)=measurePitch(
        sound, 75, 300, "Hertz")
        #make diff one for women
        
        plus.at[i, 'pitchmean'] = meanF0
        plus.at[i, 'pitchsd'] = stdevF0
        plus.at[i, 'pitchrange'] = f0range
        plus.at[i, 'dbmean'] = meanDb
        plus.at[i, 'dbsd'] = stdevDb
        plus.at[i, 'dbrange'] = dbrange

    except Exception as e:
        print(f"oh noses u have a problem in {wav_path}: {e}")

In [ ]:
plus['pitchmean'] = None
plus['pitchrange'] = None
plus['pitchsd'] = None

def measurePitch(sound, f0min, f0max, unit):
    duration = call(sound, "Get total duration")
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max)
    meanF0 = call(pitch, "Get mean", 0, 0, unit)
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit)
    minF0 = call(pitch, "Get minimum", 0, 0, unit, "Parabolic") #check this
    maxF0 = call(pitch, "Get maximum", 0, 0, unit, "Parabolic")
    f0range = maxF0 - minF0

    return duration, meanF0, stdevF0, f0range

base_dir = r'C:\Users\kasia\asp2025\textgrids'
#MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN MEN 
for i, key in enumerate(plus['SCENE']):
    try:
        wav_path = None
        for root, dirs, files in os.walk(base_dir):
            target_file = f"{key}.wav"
            if target_file in files:
                wav_path = os.path.join(root, target_file)
                break  

        if wav_path is None:
            raise FileNotFoundError(f"{key}.wav not found in subfolders")

        sound = parselmouth.Sound(wav_path)
        (duration, meanF0, stdevF0, f0range)=measurePitch(sound, 98.4, 591, "Hertz")
        #make diff one for women
        
        plus.at[i, 'pitchmean'] = meanF0
        plus.at[i, 'pitchsd'] = stdevF0
        plus.at[i, 'pitchrange'] = f0range
    except Exception as e:
        print(f"oh noses u have a problem in {wav_path}: {e}")

In [ ]:
plus.head(10)

counting syl

In [ ]:
# vowels: 
#1 = primary
#2 = secondary
#3 = no stress
plus['cmudict'] = None
plus['syl_count'] = None
d = cmudict.dict()

def cmudict_out(text):
    words = re.sub(r"\b(\w+)'[\w]+\b", r"\1", text.lower())
    words = re.findall(r"\b\w+\b", words)
    
    return [(word, d[word][0]) for word in words if word in d]


plus['cmudict'] = plus['SENTENCE'].astype(str).apply(cmudict_out)
plus['syl_count'] = plus['cmudict'].astype(str).apply(lambda x: len(re.findall(r'\d+', x)))
#plus = plus.drop(columns=['cmudict'])

In [ ]:
plus.head(3)